In [1]:
import numpy as np
import cv2 as cv
from imutils.video import VideoStream
import requests
import warnings
import time, glob

In [2]:
# Ignore warnings
warnings.filterwarnings("ignore")

In [3]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

real_size = 29 # mm

objp = np.zeros((4*5, 3), np.float32)
objp[:, :2] = np.mgrid[0:5, 0:4].T.reshape(-1, 2)*real_size

objpoints = []
imgpoints = []

images = glob.glob('./CalibrationImages/*.jpg')

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # Find chess board corners
    ret, corners = cv.findChessboardCorners(gray, (5, 4), None)
    
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        
        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners)
        
        # Draw and display the corners
        cv.drawChessboardCorners(img, (5, 4), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)

cv.destroyAllWindows()
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
img = cv.imread('./CalibrationImages/img1.jpg')
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
# UNDISTORT
dst = cv.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('./CalibrationImages/calibresult.png', dst)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
img = cv.imread('./CalibrationImages/img1.jpg')
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
# UNDISTORT
dst = cv.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('./CalibrationImages/calibresult.png', dst)

In [ ]:
from essentials import *

# Write a loop to access IP cam via an address

url = "https://192.168.1.64:14415/shot.jpg"
  
# While loop to continuously fetching data from the Url 
i = 0
while True: 
    tic = time.time()
    img_resp = requests.get(url, verify=False) 
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8) 
    img = cv2.imdecode(img_arr, -1) 
    img = cv2.resize(img, (640, 384))      
        
    
    all_objects = GetObjects(img)
  
    cam_loc = all_objects['Obj0']['location']
    cam_px = int(cam_loc[0][0] )
    cam_py = int(cam_loc[0][1] )
    

    paper_loc = image2Dworld3D([cam_px, cam_py])
    print(f"Cam Loc : {cam_loc}")
    print(f"Paper Loc : {paper_loc}")
    cv2.drawMarker(img, (cam_px, cam_py), (255, 0, 0))
    cv2.putText(img, f'({int(paper_loc[0]), int(paper_loc[1])})', (cam_px, cam_py), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    
    # cv2.circle(img, (200, 100), radius=3, color=(0, 255, 0), thickness=-1)
    # cv2.circle(img, (500, 100), radius=3, color=(0, 255, 0), thickness=-1)
    # cv2.circle(img, (200, 300), radius=3, color=(0, 255, 0), thickness=-1)
    # cv2.circle(img, (500, 300), radius=3, color=(0, 255, 0), thickness=-1)
    # cv2.putText(img, '(200, 100)', (200, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    # cv2.putText(img, '(500, 300)', (500, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    # cv2.putText(img, '(200, 300)', (200, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    # cv2.putText(img, '(500, 100)', (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    cv2.imshow("Android_cam", img)
    
    # Press Esc key to exit 
    if cv2.waitKey(1) == 27: 
        break
  
cv2.destroyAllWindows() 

In [25]:
import modern_robotics as mr

# Transferrring local(conveyor frame) sample points to robot coordinate frame

R_rcs2conv = np.array([[1, 0, 0], [0, -1, 0], [0, 0, 1]]) # Rotation matrix mapping rcs to conv
p_rcs2conv = np.array([-642.443, 132, -506])              # Translation vector from rcs to conv

T_rcs2conv = mr.RpToTrans(R_rcs2conv, p_rcs2conv)
conv_coords = [[263, 40, 0], [176, 35, 0], [258, 180, 0], [169, 174, 0]]
rcs_coords = []

for point in conv_coords:
    homgns_point = np.append(np.array(point), 1).T
    rcs_coords.append(np.dot(T_rcs2conv, homgns_point))